## *Reflection Agents*

- Reflection agents are AI systems designed to improve their own outputs by evaluating and critiquing their previous responses in a loop. 

- They use a reflection-style architecture where one agent generates an answer, and another agent (the reflection or critique agent) reviews that answer, points out errors or weaknesses, and suggests improvements. This process can repeat multiple times, refining the output iteratively until it meets quality standards or a stopping condition is reached.

*How Reflection Agents Work*
- Main agent: Attempts to solve the user’s task by generating an initial response.

 - Reflection (critique) agent: Reviews the main agent’s output, providing constructive feedback on accuracy, completeness, clarity, helpfulness, and safety.

 - The reflection agent’s critique is fed back to the main agent, which revises its response accordingly.

 - This loop continues for a fixed number of iterations or until the reflection agent signals no further improvements are needed.


*Example Workflow in LangGraph*
```
User input is sent to the main agent.
Main agent generates a response.
Reflection agent evaluates the response and either:
Sends critiques back to the main agent for revision, or
Approves the response and ends the loop.
The final improved response is returned to the user.
```





##  *Chains*
#### *Generation Chain & Reflect Chain* 

- Generation Chain and Reflect Chain are two complementary components used in LangChain (and LangGraph) to build AI workflows that generate content and then improve it through self-reflection.

*Generation Chain*
- The generation chain is responsible for producing an initial output based on the user’s input or task.
- It typically consists of a prompt template combined with a language model (LLM) that generates text, answers, or other content.

*Reflect Chain*
- The reflect chain acts as a critic or reviewer of the generation chain’s output.
- It uses a different prompt designed to evaluate, critique, and suggest improvements for the generated content.
- The reflect chain takes the generated output as input and returns feedback, recommendations, or corrections.
- This feedback is then fed back as input to the generation chain, enabling iterative refinement.
- The reflect chain is often implemented as a separate node in a cyclical workflow where generation and reflection alternate.




*How generate chain is created ?*

- LangChain handles the generation chain by structuring the process of generating content through a series of connected components called chains. A generation chain typically involves:

- Using a PromptTemplate to format the user input or context into a prompt suitable for the language model.

- Passing this prompt to a language model (LLM) to generate text output.

In [1]:
from dotenv import load_dotenv
import os
load_dotenv()


True

In [2]:
from langchain_mistralai.chat_models import ChatMistralAI
from typing import List, Sequence

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

# Initialize with API key
llm = ChatMistralAI(
    model="mistral-large-latest",api_key=os.getenv("MISTRAL_API_KEY")

)

llm = ChatOpenAI(model="gpt-3.5-turbo-0125",api_key=os.getenv("OPENAI_API_KEY"))
   
generation_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a writing assistant. Write a clear 5-paragraph essay. If critique is provided, revise your work."),
    MessagesPlaceholder(variable_name="messages"),
])
generate_chain = generation_prompt | llm

In [3]:
# Reflect Chain
reflection_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a teacher. Provide critique and suggestions to improve the following essay."),
    MessagesPlaceholder(variable_name="messages"),
])
reflect_chain = reflection_prompt | llm


In [4]:
def generation_node(state: Sequence[BaseMessage]):
    return generate_chain.invoke({"messages": state})


def reflection_node(messages: Sequence[BaseMessage]) -> List[BaseMessage]:
    res = reflect_chain.invoke({"messages": messages})
    return [HumanMessage(content=res.content)]

In [5]:
from langgraph.graph import MessageGraph, END
# Build the graph
builder = MessageGraph()
builder.add_node("generate", generation_node)
builder.add_node("reflect", reflection_node)
builder.set_entry_point("generate")

# Stop after 6 turns
def should_continue(state):
    if len(state) > 6:
        return END
    return "reflect"

builder.add_conditional_edges("generate", should_continue)
builder.add_edge("reflect", "generate")

graph = builder.compile()
print(graph.get_graph().draw_mermaid())


# Initial message
input_messages = {
    "messages": [
        {"role": "user", "content": "Write an essay about the importance of environmental conservation."}
    ]
}


---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	generate(generate)
	reflect(reflect)
	__end__([<p>__end__</p>]):::last
	__start__ --> generate;
	reflect --> generate;
	generate -.-> reflect;
	generate -.-> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



In [6]:
print("Hello LangGraph")
inputs = HumanMessage(content="Write an essay on - environment conservation @LangChainAI")
response = graph.invoke(inputs)

if isinstance(response, AIMessage):
    print("*******",response.content)
else:
    print("#######",response) 

Hello LangGraph
####### [HumanMessage(content='Write an essay on - environment conservation @LangChainAI', additional_kwargs={}, response_metadata={}, id='57ab98fe-d3e3-4145-ba42-d6183fbe0543'), AIMessage(content='Environment conservation is a critical issue that has gained increasing attention in recent years due to the impact of human activities on the planet. The environment encompasses all living and non-living things that surround us, and its conservation is essential for the well-being of current and future generations. This essay will discuss the importance of environment conservation, the threats it faces, and the actions that can be taken to preserve and protect it.\n\nFirst and foremost, environment conservation is crucial for maintaining the balance of ecosystems and biodiversity. Ecosystems provide essential services such as clean air, water, and food, which are vital for human survival. By conserving the environment, we can protect these services and ensure a sustainable f